In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import docplex
from docplex.mp.model import Model
rn = np.random
rn.seed(1)

# Line balancing problem


## Input Data


In [2]:
# Available operating time
T = 480  # minutes

# Precedence relations
precedence = [
    [0, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0],
]

# Required time for tasks
t = [5, 7, 6, 8, 6, 4, 3, 7]

# Possible stations
max_num_stations = 8

## Part a: Optimal objective value, when demand D = 32


#### Create a model instance

In [3]:
model = Model(name='line balancing problem')

#### Define indices

In [4]:
# When demand equal 32
D = 32

# Tasks
tasks = np.arange(0, 8)

# Stations
stations = np.arange(0, 8)

# Takt time
c = T / D

#### Define decision variables

In [5]:
# Which task is assign to which station
X = model.binary_var_matrix(tasks, stations, name='Task i to station m')

# Whether station is opened or not
Z = model.binary_var_dict(stations, name='Station opened or not')

#### Define constraints

In [6]:
# One task assigned to one station
for i in tasks:
    model.add_constraint(model.sum(X[i,m] for m in stations) == 1)
    
# Takt time not violated
for m in stations:
    model.add_constraint(model.sum(t[i] * X[i,m] for i in tasks) <= c * Z[m])
    
# Precedence relation
n_tasks = 8
for r in range(n_tasks):
    for s in range(n_tasks):
        if precedence[r][s] == 1:
            model.add_constraint(model.sum(m * X[r,m] for m in stations) <= model.sum(m * X[s,m] for m in stations))


#### Define linear expression

In [7]:
Y = model.sum(Z[m] for m in stations)

#### Define objective function

In [8]:
model.minimize(Y)

#### Solve model

In [9]:
model.solve()

docplex.mp.solution.SolveSolution(obj=4,values={Task i to station m_0_0:..

#### Print result

In [11]:
print(f'Optimal objective value (total number of opened stations) is {model.objective_value}.')

Optimal objective value (total number of opened stations) is 4.0.


In [14]:
model.solve_details.status

'integer optimal solution'

## Part b: What is the maximum demand?


In [10]:
# Solution to (2-b)
# Loop until the result is infeasible
d = 32
possible_demand = []
while model.solve_details.status == 'integer optimal solution':
    d = d+1
    c = T/d
    
    for i in tasks:
        model.add_constraint(model.sum(X[i,m] for m in stations) == 1)
    

    for m in stations:
        model.add_constraint(model.sum(t[i] * X[i,m] for i in tasks) <= c * Z[m])

    n_tasks = 8
    for r in range(n_tasks):
        for s in range(n_tasks):
            if precedence[r][s] == 1:
                model.add_constraint(model.sum(m * X[r,m] for m in stations) <= model.sum(m * X[s,m] for m in stations))
    
    Y = model.sum(Z[m] for m in stations)
    
    model.minimize(Y)
    
    model.solve()
    
    possible_demand.append(d)
    
print(possible_demand)    


[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]


In [12]:
print(f'Maximum possible demand is {max(possible_demand)}.')

Maximum possible demand is 61.


## Part c: When total demand is stochastic and follows an uniform distribution


In [13]:
# Generate a sample of size 100
sample = st.uniform.rvs(size=100, loc=25, scale=30)

# Mean of sampled data
print(f'Mean of sampled data is {sample.mean()}.')

# Coefficient of variation of sampled data
print(f'Coefficient of variation of sampled data is {sample.std() / sample.mean()}.')

Mean of sampled data is 39.57633782800437.
Coefficient of variation of sampled data is 0.2231648312694112.
